#### TEST

In [1]:
import pyspark
from pyspark import pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, \
StringType, IntegerType, TimestampType, DateType, FloatType
import time
import pathlib
from pyspark.sql.functions import col, to_date, sum, avg, max, min, \
stddev, percentile_approx,\
pandas_udf, PandasUDFType, lit, udf, collect_list, sqrt, monotonically_increasing_id, map_from_entries,\
rank, dense_rank, count, when
from pyspark.sql.window import Window

In [2]:
conf = pyspark.SparkConf().setAll([\
            ('spark.master', 'local[*]'),\
            ('spark.app.name', 'Glucose_Analysis_Spark')])\
            .set('spark.sql.shuffle.partitions', '1500')
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()        

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/19 21:22:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = StructType([
  StructField('PatientId', StringType(), True),
  StructField('Value', FloatType(), True),
  StructField('GlucoseDisplayDate', DateType(), True)
  ])

emptyRDD = spark.sparkContext.emptyRDD()
df = spark.createDataFrame(emptyRDD,schema)

df.printSchema()

root
 |-- PatientId: string (nullable = true)
 |-- Value: float (nullable = true)
 |-- GlucoseDisplayDate: date (nullable = true)



In [4]:
df = spark.read.parquet('/cephfs/stepped_glucose_data/step0_load/parquet_0_to_10/part-00000-532ee45d-8e0d-44c4-8f3b-884b22175e0f-c000.snappy.parquet')

In [10]:
df = df.withColumn('y_binary', lit(1))

In [11]:
#def pyspark_summary_statistics(self, df, spark):
# @pandas_udf(StructType([StructField('Entropy', FloatType())]), PandasUDFType.GROUPED_MAP)
# def entropy_grouped(df):
#     return feat_create_obj.entropy_extraction(df.Value)

def entropy_udf(vals):
    feat_create_obj = feat_create()
    return udf(feat_create_obj.entropy_extraction(vals), FloatType())

def poincare_udf(vals):
    feat_create_obj = feat_create()
    return udf(calculate_poincare(vals),\
               StructType([\
                   StructField('First', FloatType()),\
                   StructField('Second', FloatType()),\
                   StructField('Third', FloatType())\
               ]))

# def chunk_by_index():
#     return udf(collect_list

def create_partition_date(df, chunk_val):
    window = Window.partitionBy(df['PatientId']).orderBy(df['GlucoseDisplayTime'])
    df = df.select('*', rank().over(window).alias('index'))
    df = df.withColumn("Chunk", (df.index/chunk_val).cast(IntegerType()))

    return df


def pyspark_summary_statistics(df, \
                               daily_stats_features_lower,\
                               daily_stats_features_upper, \
                               chunk_val = 12):  

    df_added = create_partition_date(df, chunk_val)
    
    group_cols = ["PatientId", "Chunk"]

    summary_df = df_added.groupby(group_cols)\
        .agg(max('y_binary').alias('y_summary_binary'),\
             avg("Value").alias("Mean"),\
             stddev("Value").alias("Std Dev"),\
             percentile_approx("Value", .5).alias("Median"), \
             min("Value").alias("Min"),\
             max("Value").alias("Max"),\
             count(when(col("Value") < daily_stats_features_lower, 1)).alias("CountBelow"),\
             count(when(col("Value") > daily_stats_features_upper, 1)).alias("CountAbove"),\
             (count(when(col("Value") < daily_stats_features_lower, 1))/chunk_val).alias("PercentageBelow"),\
             (count(when(col("Value") > daily_stats_features_upper, 1))/chunk_val).alias("PercentageAbove")
            )

    df_added = df_added.join(summary_df, ['PatientId', 'Chunk'])
    
    return df_added

In [12]:
pyspark_summary_statistics(df, 70, 180, 12)

[336.457s][warning][gc,alloc] Executor task launch worker for task 0.0 in stage 13.0 (TID 27): Retried waiting for GCLocker too often allocating 4194306 words
23/04/19 21:27:36 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
23/04/19 21:27:37 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
[336.604s][warning][gc,alloc] Executor task launch worker for task 0.0 in stage 13.0 (TID 27): Retried waiting for GCLocker too often allocating 4194306 words
23/04/19 21:27:37 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.


+--------------------+-----+-----+-------------------+---------------------+------------------+--------+-----+----------------+------------------+------------------+------+-----+-----+----------+----------+---------------+---------------+
|           PatientId|Chunk|Value| GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|y_binary|index|y_summary_binary|              Mean|           Std Dev|Median|  Min|  Max|CountBelow|CountAbove|PercentageBelow|PercentageAbove|
+--------------------+-----+-----+-------------------+---------------------+------------------+--------+-----+----------------+------------------+------------------+------+-----+-----+----------+----------+---------------+---------------+
|++3L3PAkDvSTkWnWe...|    8|169.0|2022-02-19 01:56:07| 2022-02-19T01:56:...|        2022-02-19|       1|   96|               1|168.83333333333334| 2.823065172768233| 169.0|162.0|172.0|         0|         0|            0.0|            0.0|
|++3L3PAkDvSTkWnWe...|    8|170.0|2022-02-19

DataFrame[PatientId: string, Chunk: int, Value: float, GlucoseDisplayTime: timestamp, GlucoseDisplayTimeRaw: string, GlucoseDisplayDate: date, y_binary: int, index: int, y_summary_binary: int, Mean: double, Std Dev: double, Median: float, Min: float, Max: float, CountBelow: bigint, CountAbove: bigint, PercentageBelow: double, PercentageAbove: double]

In [ ]:
added_daily_features=df.groupby(analysis_group).apply(transform_features)

    return added_daily_features

In [ ]:
spark.stop()